# Imports

In [ ]:
import os
import pickle
import h5py
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.python.keras.applications import VGG19
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

# Load Data

In [ ]:
# Folder containing the datset
data_dir = 'dataset/processed_data'

In [ ]:
def load_data(filename, data_dir, data_type):
    h5f = h5py.File(os.path.join(data_dir, filename), 'r')
    data = h5f[data_type][:]
    h5f.close()
    return data

Load training data

In [ ]:
train_images = load_data('train_images.h5', data_dir, 'images')

In [ ]:
train_categories = load_data('train_categories.h5', data_dir, 'labels')

Load validation data

In [ ]:
val_images = load_data('val_images.h5', data_dir, 'images')

In [ ]:
val_categories = load_data('val_categories.h5', data_dir, 'labels')

Load test data

In [ ]:
test_images = load_data('test_images.h5', data_dir, 'images')

In [ ]:
test_categories = load_data('test_categories.h5', data_dir, 'labels')

Load mapping

In [ ]:
with open('dataset/coco_raw.pickle', 'rb') as file:
    coco_raw = pickle.load(file)
id_category = coco_raw['id_category']

In [ ]:
num_classes = len(id_category)
num_classes

In [ ]:
num_images_train = train_images.shape[0]
num_images_train

### Display a sample image

In [ ]:
# display image
idx = 10

for category, value in enumerate(train_categories[idx]):
    if value != 0:
        print(id_category[category])

plt.imshow(train_images[idx])
plt.show()

# Define Model

In [ ]:
# Download VGG19 model along with the fully-connected layers
model = VGG19(include_top=True, weights='imagenet')
model.summary()

In [ ]:
# Extract the last layer from the last convolutional block
conv_layer = model.get_layer('block5_pool')

In [ ]:
conv_layer.output

In [ ]:
# Create model
conv_model = Model(inputs=model.input, outputs=conv_layer.output)

In [ ]:
# Start a new Keras Sequential model
image_model = Sequential()

# Add the convolutional part of the VGG19 model
image_model.add(conv_model)

# Flatten the output of the VGG19 model because it is from a
# convolutional layer
image_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG19 model has
# recognized in the image.
image_model.add(Dense(1024, activation='relu', kernel_regularizer=regularizers.l1(0.01)))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
# image_model.add(Dropout(0.5))

# Add the final layer for the actual classification
image_model.add(Dense(num_classes, activation='sigmoid'))

In [ ]:
# Set the VGG19 layers to be non-trainable
conv_model.trainable = False
for layer in conv_model.layers:
    layer.trainable = False

In [ ]:
image_model.summary()

In [ ]:
# Compile the model
optimizer = Adam(lr=1e-3)

image_model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

In [ ]:
weights_dir = 'weights'
if not os.path.exists(weights_dir):
    os.mkdir(weights_dir)

path_checkpoint = os.path.join(weights_dir, 'checkpoint.keras')

# set model callbacks
tb = TensorBoard(log_dir=os.path.join(weights_dir, 'tensorboard-logs'), histogram_freq=0, write_graph=False)
checkpoint = ModelCheckpoint(filepath=path_checkpoint, verbose=1, save_weights_only=True)
# early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

callbacks = [tb, checkpoint]

# Train Model

In [ ]:
# Load previously trained model weights
try:
    image_model.load_weights(path_checkpoint)
    print('Checkpoint loaded.')
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

In [ ]:
image_model.fit(
    x=train_images,
    y=train_categories,
    batch_size=128,
    epochs=100,
    callbacks=callbacks,
    validation_data=(val_images, val_categories)
)

# Test Model

In [ ]:
def get_predictions(image, label, id_category):
    image_batch = np.expand_dims(image, axis=0)
    predictions = image_model.predict(image_batch)
    
    prediction_labels = []
    for index, prediction_probability in enumerate(predictions[0]):
        if prediction_probability > 0.5:
            prediction_labels.append(id_category[index])
    
    true_labels = []
    for index, value in enumerate(label):
        if value == 1:
            true_labels.append(id_category[index])
    
    print('True labels:', true_labels)
    print('Predictions:', prediction_labels)
    
    print('Image:')
    plt.imshow(image)
    plt.show()

In [ ]:
idx = 18
get_predictions(test_images[idx], test_categories[idx], id_category)